In [1]:
%%time

import xml.etree.ElementTree as ET
tree = ET.parse('data/enwiki-latest-pages-articles10.xml-p2336423p3046512')
root = tree.getroot()

CPU times: user 31.6 s, sys: 4.03 s, total: 35.6 s
Wall time: 37.1 s


In [119]:
pages = root.findall('./ns:page/ns:revision/ns:text', namespaces={'ns': 'http://www.mediawiki.org/xml/export-0.10/'});
titles = root.findall('./ns:page/ns:title', namespaces={'ns': 'http://www.mediawiki.org/xml/export-0.10/'});

assert len(titles) == len(pages), 1

titles_texts = [(title.text, page.text) for title, page in zip(titles, pages) if '{{coord|' in (page.text or '')]
len(titles_texts), len(pages)

((21720, 308503), 21719, 308503)

In [116]:
'Antarctica', 'Prince Charles Mountains'

In [92]:
def coordinate_normalize(N, W):
    degrees_N, minutes_N, seconds_N = N
    degrees_W, minutes_W, seconds_W = W
    
    return (degrees_N + minutes_N / 60 + seconds_N / 3600,
            degrees_W + minutes_W / 60 + seconds_W / 3600)
            
def coordinate_extract(coordinate_text):
    import re
    
    regex = r"{{coord\|(.+?)\|(.+?)\|(.+?)\|N\|(.+?)\|(.+?)\|(.+?)\|W(?:\|.+)?"
    results = re.findall(regex, coordinate_text, re.IGNORECASE)

    if results != []:
        # print(results)
        dn, mn, sn, dw, mw, sw = [float(num) for num in results[0]]
        coordinate = coordinate_normalize(N=(dn, mn, sn), W=(-dw, -mw, -sw))
        return coordinate
    else:
        regex = r"{{coord\|(.+?)\|(.+?)\|N\|(.+?)\|(.+?)\|W(?:\|.+)?"
        results = re.findall(regex, coordinate_text, re.IGNORECASE)
        
    if results != [] and '|' not in results[0]:
        dn, mn, dw, mw = [float(num) for num in results[0]]
        coordinate = coordinate_normalize(N=(dn, mn, 0), W=(-dw, -mw, 0))
        
        return coordinate
    else:
        regex = r"{{coord\|(.+?)\|+N\|(.+?)\|+W(?:\|.+)?"
        results = re.findall(regex, coordinate_text, re.IGNORECASE)
        
    if results != []:
        dn, dw = [float(num) for num in results[0]]
        coordinate = coordinate_normalize(N=(dn, 0, 0), W=(-dw, 0, 0))
    else:
        regex = r"{{coord\|(.+?)\|([.\-0-9]+)"
        results = re.findall(regex, coordinate_text, re.IGNORECASE)
        
    if results != []:
        dn, dw = [float(num) for num in results[0]]
        coordinate = coordinate_normalize(N=(dn, 0, 0), W=(dw, 0, 0))
        
        return coordinate
    else:
        return []

def coordinate_try(coordinate_text):
    try:
        return coordinate_extract(coordinate_text)
    except:
        # print("E:" + coordinate_text)
        return None

In [94]:
import re
def coordinates_parse(text):
    return re.findall(r"{{coord\|.+?}}", text)

coordinates_all = sum([coordinates_parse(text) for title, text in titles_texts], [])
# coordinates_all = [coordinate_data for coordinate_text in coordinates_all
#                    if coordinate_data := coordinate_try(coordinate_text)]

print(len(coordinates_all), len(set(coordinates_all)))
coordinates_all = [coordinate_try(coordinate_text) for coordinate_text in coordinates_all if coordinate_try(coordinate_text)]

len(coordinates_all), len(set(coordinates_all))

coordinates_all = set(coordinates_all)

coordinates_master_list = list(coordinates_all)
coordinates_indices_dict = {coordinate: index for index, coordinate in enumerate(coordinates_master_list)}

def coordinate_index(coordinate_text):
    if coordinate_text:
        coordinate = coordinate_try(coordinate_text)

        if coordinate:
            return coordinates_indices_dict[coordinate]

31467 30611


In [90]:
coordinates_all = sum([[(coordinate, index) for coordinate in coordinates_parse(text)]
                       for index, (title, text) in enumerate(titles_texts)], [])

coordinates_articles_index_pairs = [(coordinate_index(coordinate_text), index)
                                    for coordinate_text, index in coordinates_all if coordinate_try(coordinate_text)]

coordinates_articles_index_array = list(map(lambda _: set(), range(len(coordinates_master_list))))
for coordinate_index_, article_index in coordinates_articles_index_pairs:
    coordinates_articles_index_array[coordinate_index_].add(article_index)

with open('database/coordinates_articles_index_array.json', 'w') as file:
    file.write(json.dumps(list(map(list, coordinates_articles_index_array))))

titles_all = [title for title, text in titles_texts]

with open('database/article_titles_array.json', 'w') as file:
    file.write(json.dumps(titles_all, separators=(',', ':')))

In [98]:
%%time

# https://dev.mysql.com/doc/refman/5.7/en/fulltext-stopwords.html#fulltext-stopwords-stopwords-for-myisam-search-indexes

stopwords = """
a's           able          about         above         according
accordingly   across        actually      after         afterwards
again         against       ain't         all           allow
allows        almost        alone         along         already
also          although      always        am            among
amongst       an            and           another       any
anybody       anyhow        anyone        anything      anyway
anyways       anywhere      apart         appear        appreciate
appropriate   are           aren't        around        as
aside         ask           asking        associated    at
available     away          awfully       be            became
because       become        becomes       becoming      been
before        beforehand    behind        being         believe
below         beside        besides       best          better
between       beyond        both          brief         but
by            c'mon         c's           came          can
can't         cannot        cant          cause         causes
certain       certainly     changes       clearly       co
com           come          comes         concerning    consequently
consider      considering   contain       containing    contains
corresponding could         couldn't      course        currently
definitely    described     despite       did           didn't
different     do            does          doesn't       doing
don't         done          down          downwards     during
each          edu           eg            eight         either
else          elsewhere     enough        entirely      especially
et            etc           even          ever          every
everybody     everyone      everything    everywhere    ex
exactly       example       except        far           few
fifth         first         five          followed      following
follows       for           former        formerly      forth
four          from          further       furthermore   get
gets          getting       given         gives         go
goes          going         gone          got           gotten
greetings     had           hadn't        happens       hardly
has           hasn't        have          haven't       having
he            he's          hello         help          hence
her           here          here's        hereafter     hereby
herein        hereupon      hers          herself       hi
him           himself       his           hither        hopefully
how           howbeit       however       i'd           i'll
i'm           i've          ie            if            ignored
immediate     in            inasmuch      inc           indeed
indicate      indicated     indicates     inner         insofar
instead       into          inward        is            isn't
it            it'd          it'll         it's          its
itself        just          keep          keeps         kept
know          known         knows         last          lately
later         latter        latterly      least         less
lest          let           let's         like          liked
likely        little        look          looking       looks
ltd           mainly        many          may           maybe
me            mean          meanwhile     merely        might
more          moreover      most          mostly        much
must          my            myself        name          namely
nd            near          nearly        necessary     need
needs         neither       never         nevertheless  new
next          nine          no            nobody        non
none          noone         nor           normally      not
nothing       novel         now           nowhere       obviously
of            off           often         oh            ok
okay          old           on            once          one
ones          only          onto          or            other
others        otherwise     ought         our           ours
ourselves     out           outside       over          overall
own           particular    particularly  per           perhaps
placed        please        plus          possible      presumably
probably      provides      que           quite         qv
rather        rd            re            really        reasonably
regarding     regardless    regards       relatively    respectively
right         said          same          saw           say
saying        says          second        secondly      see
seeing        seem          seemed        seeming       seems
seen          self          selves        sensible      sent
serious       seriously     seven         several       shall
she           should        shouldn't     since         six
so            some          somebody      somehow       someone
something     sometime      sometimes     somewhat      somewhere
soon          sorry         specified     specify       specifying
still         sub           such          sup           sure
t's           take          taken         tell          tends
th            than          thank         thanks        thanx
that          that's        thats         the           their
theirs        them          themselves    then          thence
there         there's       thereafter    thereby       therefore
therein       theres        thereupon     these         they
they'd        they'll       they're       they've       think
third         this          thorough      thoroughly    those
though        three         through       throughout    thru
thus          to            together      too           took
toward        towards       tried         tries         truly
try           trying        twice         two           un
under         unfortunately unless        unlikely      until
unto          up            upon          us            use
used          useful        uses          using         usually
value         various       very          via           viz
vs            want          wants         was           wasn't
way           we            we'd          we'll         we're
we've         welcome       well          went          were
weren't       what          what's        whatever      when
whence        whenever      where         where's       whereafter
whereas       whereby       wherein       whereupon     wherever
whether       which         while         whither       who
who's         whoever       whole         whom          whose
why           will          willing       wish          with
within        without       won't         wonder        would
wouldn't      yes           yet           you           you'd
you'll        you're        you've        your          yours
yourself      yourselves    zero"""

stopwords = stopwords.split()

def wikitext_clean(wikitext):
    curly_counter = 0
    square_counter = 0

    output_characters = ['']*len(text)
    character_pointer = 0

    link_record = []

    for c in text:
        if c == '{':
            curly_counter += 1

        if square_counter == 2:
            link_record.append(c)

        if curly_counter == 0 and c == '[':
            square_counter += 1

        if curly_counter == 0 and square_counter == 0:
            output_characters[character_pointer] = c
            character_pointer += 1

        if c == '}':
            curly_counter -= 1

        if curly_counter == 0 and square_counter == 2 and c == '|':
            link_record = []

        if curly_counter == 0 and c == ']':
            square_counter -= 1

            if square_counter == 0:
                output_characters[character_pointer] = "~>" + ''.join(link_record)[:-1] + "<~"
                character_pointer += 1
                link_record = []

    return ''.join(output_characters)

alphabet = list("0123456789") + list(map(chr, range(97, 97+26)))
alphabet = list(map(chr, range(97, 97+26)))

print(f'alphabet: {" ".join(alphabet)}')

coordinates_word_index = {}
titles_word_index = {}

for index, (title, text) in enumerate(titles_texts):
    cleaned = wikitext_clean(text).lower()

    words = ''.join([c if c in alphabet else ' ' for c in cleaned]).split()
    words_destopped = set(words).difference(stopwords)
    words_deshorted = set(word for word in words_destopped if len(word) >= 3)

    len(words), len(set(words)), len(words_destopped), len(words_deshorted)
    words_deshorted

    coordinate_indices = [coordinate_index(coordinate_text) for coordinate_text in coordinates_parse(text)
                          if coordinate_index(coordinate_text)]

    for word in words_deshorted:
        coordinates_word_index[word] = coordinates_word_index.get(word, set())
        coordinates_word_index[word].update(coordinate_indices)

        titles_word_index[word] = titles_word_index.get(word, set())
        titles_word_index[word].add(index)

alphabet: a b c d e f g h i j k l m n o p q r s t u v w x y z
CPU times: user 1min 49s, sys: 9.36 s, total: 1min 58s
Wall time: 2min 7s


In [112]:
import json

with open('database/coordinates_master_list.json', 'w') as file:
    file.write(json.dumps(coordinates_master_list, separators=(',', ':')))

with open('database/coordinates_word_index.json', 'w') as file:
    file.write(json.dumps({key: list(value) for key, value in coordinates_word_index.items()}, separators=(',', ':')))

with open('database/titles_word_index.json', 'w') as file:
    file.write(json.dumps({key: list(value) for key, value in titles_word_index.items()}, separators=(',', ':')))

In [ ]:
'api.php?action=query&list=search&srsearch=Craig%20Noone&format=json'

In [106]:
%%time
len(index.get('ghost')), len(index.get('town')), len(index['ghost'].intersection(index['town']).intersection(index['mining']))
[coordinates_master_list[i] for i in index['ghost'].intersection(index['town']).intersection(index['mining'])]

CPU times: user 98 µs, sys: 0 ns, total: 98 µs
Wall time: 102 µs


[(34.91916666666666, -115.06277777777777),
 (38.17027777777778, -119.1975),
 (50.72555555555556, -121.28055555555555),
 (38.1701984, 119.1973749),
 (42.468333333333334, -108.79972222222221),
 (44.464444444444446, -109.61361111111111),
 (35.0, 59.0),
 (44.397777777777776, -114.31166666666667),
 (20.0, 20.0),
 (4.3116, -52.137),
 (53.63249, -1.37556),
 (39.59388888888889, -121.52944444444444),
 (44.96666666666667, -78.25),
 (39.556666666666665, -121.4413888888889)]

In [ ]:
(np.arange(3**5) / (3**1) % 3).astype('int')

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1,
       1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2,
       2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0,
       1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2,
       2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0,
       0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1,
       2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0,
       0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1,
       1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2,
       0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1,
       1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2,
       2])

In [227]:
max_power = 3
indices = np.stack([(np.arange(3**max_power) / (3**power) % 3).astype(np.int) for power in range(3)]).transpose()[:,::-1]
indices

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2]])

In [229]:
base = len(alphabet)
max_power = 3

indices = np.stack([(np.arange(base**max_power) / (base**power) % base).astype(np.int) for power in range(max_power)]).transpose()[:,::-1]

code_to_tuple_list = list(map(''.join, np.array(alphabet)[indices].tolist()))
tuple_to_code_dict = {tuple_: code for code, tuple_ in enumerate(code_to_tuple_list)}

In [253]:
n_sequences = lambda w, n: [w[i:i+n] for i in range(len(w) - (n - 1))]

eleven = {}
for tuple_ in set(sum([n_sequences(word, 11) for word in words], [])):
    eleven[tuple_] = eleven.get(tuple_, []) + [page_index]

In [255]:
eleven

{'commemorati': [5],
 'proximately': [5],
 'dditionally': [5],
 'additionall': [5],
 'approximate': [5],
 'admasambhav': [5],
 'corporation': [5],
 'pproximatel': [5],
 'bodhisattva': [5],
 'respectivel': [5],
 'espectively': [5],
 'mmemorating': [5],
 'padmasambha': [5],
 'overlooking': [5],
 'ommemoratin': [5],
 'celebrating': [5],
 'anniversary': [5],
 'constructed': [5],
 'dmasambhava': [5],
 'constructio': [5],
 'onstruction': [5],
 'accommodate': [5]}